# Pruned NH-AMPCs 

## Imports

In [ ]:
import os
import numpy as np
import torch

from itertools import product
from pprint import pprint
from tqdm.notebook import tqdm_notebook

In [ ]:
from src.parameters import NH_AMPC_Param
from src.generate_trajs import generate_NH_AMPC_trajs

## Settings

In [ ]:
RESULTS_DIR = os.path.abspath('Results')
MPC_DATASETS_DIR = os.path.join(RESULTS_DIR, 'MPC_data_gen')
NNS_DIR = os.path.join(RESULTS_DIR, 'Trained_Networks')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'NH_AMPC_results')
PRUNED_NNS_DIR = os.path.join(RESULTS_DIR, 'Prun_Networks')

MAKE_TEST_RESULTS = False

if MAKE_TEST_RESULTS:
    MPC_RESULTS_DIR += '_Test'


NUM_SAMPLES = 30_000
# USE_TOP_NNS = 5

NUM_NNS = 50

USE_CUDA = False

MPC_PARAM_DICT = {
    'T_sim': 5, # length of the closed-loop simulation (in seconds)
}

HORIZON_OPTIONS = [tup for tup in product(
    (8, ),                                      # N_MPC
    (22, ),                                     # N_NN -> if USE_BEGINING_OF_DATASET != 'begin, it has to be 70-max(N_MPCs)
    (70, ),                                     # N_DATASET
    (5, ),                                      # TRAIN_DATASET_VERSION
    (6, ),                                      # TEST_DATASET_VERSION
    ('fixed', ),                                # USE_BEGINING_OF_DATASET ('begin', 'fixed', '') if '' use DS_FEATURE of N_MPC
    (8, ),                                      # DS_FEATURE_IF_FIXED
    ('RTI_PCHPIPM_DISCRETE', ),                 # DATASET_NAME_ADD ('RTI_PCHPIPM_DISCRETE', 'RTI_PCHPIPM_DISCRETE_50ITER', 'RTI_PCHPIPM_ROBUST_DISCRETE')
    (64, ),                                     # HIDDEN_NEURONS
    (16, ),                           # END_HIDDEN_SIZES (left from pruning)
)]

In [ ]:
pprint(HORIZON_OPTIONS)
print(NNS_DIR)
print(PRUNED_NNS_DIR)

In [ ]:
# parameters needed for neural horizon acados MPC
NH_AMPC_PARAMS = [
    NH_AMPC_Param(
        # Param
        N_MPC = N_MPC, 
        N = N_NN+N_MPC,

        # Dataset stuff
        N_DS = N_DSET, 
        TRAIN_V_DS = TRAIN_DATASET_VERSION, 
        TEST_V_DS = TEST_DATASET_VERSION, 
        DS_begin = USE_BEGIN,
        DS_samples = NUM_SAMPLES,
        DS_opts_name = DS_OPT_NAME,
        DS_feature = DS_FEATURES,

        # NN stuff
        V_NN = NN_VERSION,
        N_hidden = N_HIDDEN,
        N_hidden_end = END_N_SIZES,

        # Param
        **MPC_PARAM_DICT
    ) for NN_VERSION in range(NUM_NNS) \
        for N_MPC, N_NN, N_DSET, TRAIN_DATASET_VERSION, TEST_DATASET_VERSION, \
            USE_BEGIN, DS_FEATURES, DS_OPT_NAME, N_HIDDEN, END_N_SIZES in HORIZON_OPTIONS \
]

### Acados options

In [ ]:
# qp_solver
# ---------
#       'FULL_CONDENSING_QPOASES', 'FULL_CONDENSING_HPIPM', 'FULL_CONDENSING_DAQP',
#       'PARTIAL_CONDENSING_HPIPM', 'PARTIAL_CONDENSING_QPDUNES', 'PARTIAL_CONDENSING_OSQP'
# hessian_approx
# --------------
#       'GAUSS_NEWTON', 'EXACT'
# integrator_type
# ---------------
#       'ERK', 'IRK', 'DISCRETE', 'LIFTED_IRK', 'GNSF'
# nlp_solver_type
# ---------------
#       'SQP_RTI', 'SQP'
# regularize_method
# -----------------
#       'NO_REGULARIZE', 'MIRROR', 'PROJECT', 'PROJECT_REDUC_HESS', 'CONVEXIFY'
# hpipm_mode
# ----------
#       'BALANCE', 'SPEED_ABS', 'SPEED', 'ROBUST'
# collocation_type
# ----------------
#       'GAUSS_RADAU_IIA', 'GAUSS_LEGENDRE'
# globalization
# -------------
#       'FIXED_STEP', 'MERIT_BACKTRACKING'
# nlp_solver_tol_stat
# -------------------
#       Default: 1e-6
# nlp_solver_max_iter
# -------------------
#       Default: 100
# as_rti_iter
# -----------
#       Default: 1
# as_rti_level
# ------------
#       Default: 4
# 
#############################################
# use_iter_rti_impl
# -----------------
#       Default = False
# use_initial_guesses
# -----------------
#       Default = False
# max_rti_iters
# -----------------
#       Default = 10
# rti_tol
# -----------------
#       Default = 1e-4



SOLVER_OPTIONS = {
    f'ASRTID_FCH': dict(
        qp_solver='FULL_CONDENSING_HPIPM', 
        integrator_type='DISCRETE', 
        nlp_solver_type='SQP_RTI',
        as_rti_iter=3,
        as_rti_level=3,
        nlp_solver_tol_stat=1e-6,
        nlp_solver_max_iter=3,
    )
}

## Load Networks and generate NH-AMPC trajectory

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() and USE_CUDA else 'cpu') 
dtype = torch.float32
print(device)

In [ ]:
with tqdm_notebook(total=len(SOLVER_OPTIONS)*len(NH_AMPC_PARAMS), unit='MPC', desc='Get trajectories: ') as tqdm_handle:
    for nh_ampc_param in NH_AMPC_PARAMS:
        for solver_option in SOLVER_OPTIONS.items():
            name = f'{solver_option[0]} {nh_ampc_param.N_MPC}M_{nh_ampc_param.N_NN}N {nh_ampc_param.N_hidden}Nh_{nh_ampc_param.V_NN}v' \
                if nh_ampc_param.N_hidden_end is None else \
                f'{solver_option[0]} {nh_ampc_param.N_MPC}M_{nh_ampc_param.N_NN}N {nh_ampc_param.N_hidden}Nh_{nh_ampc_param.N_hidden_end}NhP_{nh_ampc_param.V_NN}v'
            tqdm_handle.set_description_str(f'Get trajectory of:\n{name}')
            
            results = generate_NH_AMPC_trajs(nh_ampc_param, solver_option, MPC_RESULTS_DIR, NNS_DIR, MPC_DATASETS_DIR, device, dtype)

            tqdm_handle.update(1)
            tqdm_handle.set_postfix_str(f'\n{name}:\nCost:{results.Cost:.2f}, Time:{np.mean(results.Time)*1e3:.2f}ms, ATime:{np.mean(results.Acados_Time)*1e3:.2f}ms')